![head.png](figures/head.jpg)

# Financial Data Analytics in Python

**Prof. Dr. Fabian Woebbeking**</br>
Assistant Professor of Financial Economics

IWH - Leibniz Institute for Economic Research</br>
MLU - Martin Luther University Halle-Wittenberg

fabian.woebbeking@iwh-halle.de

In [4]:
# Packages used in this notebook
import django
import requests

# Django (web-framework)

Have a look at: [https://www.djangoproject.com/](https://www.djangoproject.com/)

Django is a high-level Python web framework that encourages rapid development and clean, pragmatic design. It's known for its simplicity and robustness, which makes it ideal for web projects with a focus on financial data analytics. A few known users of Django: Instagram, Disqus, Pinterest.

1. **Robustness**: Django's built-in features support quick and efficient web application development.
2. **Admin Interface**: Comes with a built-in admin module, which you can use to visualize and manage your data.
3. **Scalable**: Django applications can handle large volumes of data and traffic, which is essential for financial analysis tools.
4. **Security**: Django provides built-in protection against many security threats like SQL injection, cross-site scripting, cross-site request forgery, etc.



## Setting up a new Django project

We'll set up a basic Django project called "myproject" to demonstrate how you can start integrating Django into your financial analytics projects. Assuming that you have the django package installed, the follwing commands refer to the command line:

```Bash
# 1. Create a new project
django-admin startproject myproject
# Make sure to execute 2. and 3. from the myproject folder
cd myproject

# 2. Migrate
python manage.py migrate

# 3. Start the server
python manage.py runserver
```

This is how the myproject folder should look like:

```Bash
myproject/
│
├── myproject/
│   ├── __init__.py    # Tells Python that this directory should be considered a Python package
│   ├── settings.py    # Contains settings/configuration for this Django project
│   ├── urls.py        # The URL declarations for this Django project; a "table of contents" of your Django-powered site
│   ├── wsgi.py        # An entry-point for WSGI-compatible web servers to serve your project
│   └── asgi.py        # An entry-point for ASGI-compatible web servers to serve your project asynchronously
│
├── db.sqlite3         # This is the project data base, which will be updated by migrate
└── manage.py          # A command-line utility that lets you interact with this Django project
```

All these files have been created by the `startproject` command. With one exception being the **db.sqlite3** file, which has been created by the `migrate` command.

## Dajngo apps

The project is merely a shell, the magic happens within one or multiple apps that we add to the project.

```Bash
python manage.py startapp myapp
```

The folder should now look like this:

```Bash
myproject/
│
├── myproject/
│   ├── __init__.py    # Marks this directory as a Python package
│   ├── settings.py    # Contains settings/configuration for this Django project
│   ├── urls.py        # The URL declarations for this Django project
│   ├── wsgi.py        # Entry-point for WSGI-compatible web servers
│   └── asgi.py        # Entry-point for ASGI-compatible web servers
│
├── myapp/
│   ├── migrations/
│   │   ├── __init__.py # Marks migrations as a Python package
│   │   └── ...         # Other migration files
│   ├── __init__.py     # Marks myapp directory as a Python package
│   ├── admin.py        # Admin panel configuration for myapp
│   ├── apps.py         # Application-specific configuration
│   ├── models.py       # Database models
│   ├── tests.py        # Test cases for myapp
│   ├── views.py        # Views for handling requests
│   └── urls.py         # URL declarations specific to myapp (optional)
│
├── db.sqlite3         # This is the project data base
└── manage.py          # A command-line utility that lets you interact with this Django project
```

After creating the app, we have to register it in settings.py:

```Python
# ...
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'myapp',  # Add this line
]
# ...
```

## Django's components

The folder structure above corresponds to this:

1. Project and Apps
    - **Project:** The overall configuration and container for your web application, which includes settings for the database, Django-specific options, and application-specific settings.
    - **Apps:** Modular components of a project. Each app is a web application that performs a specific function (e.g., a blog or a database of records). Apps can be reused across multiple projects.

2. Settings
    - Located in `settings.py`, this file includes configurations like database setup, debug mode, static files settings, middleware configurations, and more.

3. URL Dispatcher
    - URL patterns are defined in `urls.py`. Django uses these patterns to match the client request URL to the appropriate view.

4. **Models**
    - Defined in `models.py`, models are Python classes that define the data structure. Django translate these models into database tables.

5. **Views**
    - Located in `views.py`, views are functions or classes that handle requests and return responses. They interact with models to fetch and manage data, and delegate formatting to templates.

6. **Templates**
    - HTML files that use Python-like expressions for dynamic content generation, managed through Django’s template engine.

7. Forms
    - Defined in `forms.py`, forms handle and validate data submitted via HTTP POST requests, simplifying the creation and reuse of web forms.

8. Admin
    - An automatic admin interface provided by Django to manage data in the database using a user-friendly interface.

9. Middleware
    - Middleware are hooks into Django's request/response processing, acting as a low-level plugin system for altering input or output globally.

10. Migrations
    - Located in `migrations/` folders within each app, migrations are Django’s way of propagating changes from models to the database schema.

Each component of Django is designed to ensure the framework remains robust, scalable, and maintainable.


## Cloning a Django project

```Bash
# Clone the project
git clone https://github.com/cafawo/StudySnips.git
# Locate the project folder
cd myproject
```

Let us ensure that all the required packages are installed. To do so, we will create a new conda environment called snipsenv. Please note that we are using a requirements.txt file in pip format, as an alternative to the requirements.yml file that can be created using conda.

```Bash
conda create -c conda-forge --name snipsenv --file requirements.txt
conda activate snipsenv
```

Let's get back to Dajngo:

```Bash
# 2. Migrate
python manage.py migrate

# 3. Start the server
python manage.py runserver

```

Make sure that the server is running and head over to the landing page, e.g. [http://127.0.0.1:8000/](http://127.0.0.1:8000/)

## Dajngo admin

Django comes with a build in admin feature. To access the admin feature, you have to create a user (or superuser ... something like an admin).

```Bash
python manage.py createsuperuser
```

Make sure that the server is running and head over to the admin page, e.g. [http://127.0.0.1:8000/admin/](http://127.0.0.1:8000/admin/). In the specific use case of StudySnips, the admin page allows us to create a class and add snips, which can then be claimed by students.

## Building an API with Django

Let's say we would like to return data on the number of claimed snips. This data could later be displayed in a chart or analyzed in python. 

We need two components:

1. **snips/views.py**, here we will add a function to obtain the data:
    ```Python
    def api_chart_data(request):
        # Filter the Snips where student_id is not empty or null, then group by date
        data = (Snip.objects
                    .filter(student_id__isnull=False, student_id__gt='')
                    .annotate(date=TruncDate('updated_at'))
                    .values('date')
                    .annotate(count=Count('id'))
                    .order_by('date'))
        # Format the data for the chart (date in 'YYYY-MM-DD' format and the count)
        chart_data = [{'date': entry['date'].isoformat(), 'count': entry['count']} for entry in data]
        return JsonResponse(chart_data, safe=False)
    ```

2. **snips/urls.py**, here we will point the api endpoint 'api/chart' to the aforementioned function:
    ```Python
    urlpatterns = [
        # ...
        path('api/chart', views.api_chart_data, name='chart_data'),
        # ...
    ]
    ```

Have a look at our API endpoint [http://127.0.0.1:8000/api/chart](http://127.0.0.1:8000/api/chart), you should see a JSON object with the counts per day. Of course, this data can also be used in python (if the server is running!), e.g.:

In [5]:
# URL to our Django server hosting the API
url = 'http://127.0.0.1:8000/api/chart'

# Make a GET request to the API
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    print("Data retrieved successfully!")
    print(data)
else:
    print("Failed to retrieve data. Status code:", response.status_code)

Data retrieved successfully!
[{'date': '2024-06-04', 'count': 2}]


## Displaying data using Django and Chart.js

Have a look at: [https://www.chartjs.org/](https://www.chartjs.org/)

You could of course try to display matplotlib files, however, Chart.js creates pretty and interactive charts, see e.g. [https://www.thecvx.com/](https://www.thecvx.com/).

Some more additions to the project:

1. **snips/templates/snips/chart.html**, this time we have to create a whole html template, including all the java script (Chart.js) code. This is the interesting part:
    ```js
    //...
    fetch('/api/chart')
        .then(response => response.json())
        .then(data => {
            const dates = data.map(item => item.date);
            const counts = data.map(item => item.count);
            const chart = new Chart(ctx, {
                type: 'bar', // Changed to 'bar' chart type
                data: {
                    labels: dates,
                    datasets: [{
                        label: 'Claimed Snips per Day',
                        data: counts,
                        backgroundColor: 'rgba(75, 192, 192, 0.2)', // Light blue fill
                        borderColor: 'rgb(75, 192, 192)', // Solid blue border
                        borderWidth: 1
                    }]
    //...
    ```

2. **snips/views.py**, not much to do here, just point at the template:
    ```Python
    def chart_page(request):
        return render(request, 'snips/chart.html')
    ```

3. **snips/urls.py**, here we will point to the aforementioned function:
    ```Python
    urlpatterns = [
        # ...
        path('chart/', views.chart_page, name='chart_page'),
        # ...
    ]
    ```